<a href="https://colab.research.google.com/github/wenjunsun/Covid-19-analysis-with-uw-ubicomp/blob/master/data_and_pre-processing/data_preparing_for_propensity_score_matching.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Aggregate before/after first case data of each county, and add a column of whether this county has issued Shelter in place

In [4]:
cd drive/My\ Drive/data

/content/drive/My Drive/data


In [1]:
import pandas as pd
import numpy as np

In [5]:
ls

agg_social_dist_2.csv
agg_social_dist.csv
agg_social_dist_without_low_device_count.csv
county_data.csv
county_data_with_and_without_SIP.csv
county_data_with_covariates.csv
county_data_with_reduced_covariates.csv
data_preparing_for_propensity_score_matching.ipynb
filter_low_device_count_data.ipynb
merged_data.csv
shelter_in_place_by_counties.csv
social_dist_all_trimmed.csv


In [6]:
data = pd.read_csv("agg_social_dist_without_low_device_count.csv")

In [7]:
data.head()

,date_range_start,state,state_code,cnamelong,county_code,days_since_first_case,days_since_shelter,candidate_device_count,device_count,completely_home_device_count,part_time_work_behavior_devices,full_time_work_behavior_devices,delivery_behavior_devices,median_home_dwell_time,median_non_home_dwell_time,median_percentage_time_home,distance_traveled_from_home,Date - first case,Date - shelter in place
0,2020-01-01T00:00:00+10:00,66.0,GU,Guam,66010,-75.0,NaN,0.0,310,98,41,72,0.0,395.312903,NaN,NaN,3.722158e+06,2020-03-16,NaN
1,2020-01-01T00:00:00+10:00,69.0,MP,Saipan Municipality,69110,-90.0,NaN,0.0,303,121,33,35,0.0,779.036304,NaN,NaN,NaN,2020-03-31,NaN
2,2020-01-01T00:00:00-04:00,72.0,PR,Adjuntas Municipio,72001,-75.0,-74.0,0.0,241,115,14,19,0.0,890.377593,NaN,NaN,1.405979e+05,2020-03-16,2020-03-15 00:00:00
3,2020-01-01T00:00:00-04:00,72.0,PR,Aguada Municipio,72003,-75.0,-74.0,0.0,697,327,40,59,0.0,775.674319,NaN,NaN,6.013874e+04,2020-03-16,2020-03-15 00:00:00
4,2020-01-01T00:00:00-04:00,72.0,PR,Aguadilla Municipio,72005,-75.0,-74.0,0.0,1103,518,55,98,0.0,740.937443,NaN,NaN,8.579466e+03,2020-03-16,2020-03-15 00:00:00


In [8]:
data.shape

(381904, 19)

In [9]:
data.isnull().sum()

date_range_start                        0
state                                 127
state_code                            127
cnamelong                             127
county_code                             0
days_since_first_case               10714
days_since_shelter                  54475
candidate_device_count                  0
device_count                            0
completely_home_device_count            0
part_time_work_behavior_devices         0
full_time_work_behavior_devices         0
delivery_behavior_devices               0
median_home_dwell_time                  0
median_non_home_dwell_time         345638
median_percentage_time_home        345638
distance_traveled_from_home           803
Date - first case                   10714
Date - shelter in place             54475
dtype: int64

We want to aggregate data on before/after first covid case, but we have $10704$ null values in days_since_first_case?? \\
Let's drop these rows for now...

In [10]:
data.dropna(subset=['days_since_first_case'], inplace = True)

In [11]:
data.shape

(371190, 19)

In [12]:
data.isnull().sum()

date_range_start                        0
state                                   0
state_code                              0
cnamelong                               0
county_code                             0
days_since_first_case                   0
days_since_shelter                  51898
candidate_device_count                  0
device_count                            0
completely_home_device_count            0
part_time_work_behavior_devices         0
full_time_work_behavior_devices         0
delivery_behavior_devices               0
median_home_dwell_time                  0
median_non_home_dwell_time         336000
median_percentage_time_home        336000
distance_traveled_from_home           720
Date - first case                       0
Date - shelter in place             51898
dtype: int64

Now let's aggregate all the data of counties before / after first covid case

In [13]:
before_covid_data = data[data['days_since_first_case'] < 0]
after_covid_data = data[data['days_since_first_case'] >= 0]

In [14]:
before_covid_data.head()

,date_range_start,state,state_code,cnamelong,county_code,days_since_first_case,days_since_shelter,candidate_device_count,device_count,completely_home_device_count,part_time_work_behavior_devices,full_time_work_behavior_devices,delivery_behavior_devices,median_home_dwell_time,median_non_home_dwell_time,median_percentage_time_home,distance_traveled_from_home,Date - first case,Date - shelter in place
0,2020-01-01T00:00:00+10:00,66.0,GU,Guam,66010,-75.0,NaN,0.0,310,98,41,72,0.0,395.312903,NaN,NaN,3.722158e+06,2020-03-16,NaN
1,2020-01-01T00:00:00+10:00,69.0,MP,Saipan Municipality,69110,-90.0,NaN,0.0,303,121,33,35,0.0,779.036304,NaN,NaN,NaN,2020-03-31,NaN
2,2020-01-01T00:00:00-04:00,72.0,PR,Adjuntas Municipio,72001,-75.0,-74.0,0.0,241,115,14,19,0.0,890.377593,NaN,NaN,1.405979e+05,2020-03-16,2020-03-15 00:00:00
3,2020-01-01T00:00:00-04:00,72.0,PR,Aguada Municipio,72003,-75.0,-74.0,0.0,697,327,40,59,0.0,775.674319,NaN,NaN,6.013874e+04,2020-03-16,2020-03-15 00:00:00
4,2020-01-01T00:00:00-04:00,72.0,PR,Aguadilla Municipio,72005,-75.0,-74.0,0.0,1103,518,55,98,0.0,740.937443,NaN,NaN,8.579466e+03,2020-03-16,2020-03-15 00:00:00


In [18]:
before_covid_data.drop("date_range_start", axis = 1, inplace=True)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [19]:
after_covid_data.drop("date_range_start", axis = 1, inplace=True)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [20]:
before_covid_data.head()

,state,state_code,cnamelong,county_code,days_since_first_case,days_since_shelter,candidate_device_count,device_count,completely_home_device_count,part_time_work_behavior_devices,full_time_work_behavior_devices,delivery_behavior_devices,median_home_dwell_time,median_non_home_dwell_time,median_percentage_time_home,distance_traveled_from_home,Date - first case,Date - shelter in place
0,66.0,GU,Guam,66010,-75.0,NaN,0.0,310,98,41,72,0.0,395.312903,NaN,NaN,3.722158e+06,2020-03-16,NaN
1,69.0,MP,Saipan Municipality,69110,-90.0,NaN,0.0,303,121,33,35,0.0,779.036304,NaN,NaN,NaN,2020-03-31,NaN
2,72.0,PR,Adjuntas Municipio,72001,-75.0,-74.0,0.0,241,115,14,19,0.0,890.377593,NaN,NaN,1.405979e+05,2020-03-16,2020-03-15 00:00:00
3,72.0,PR,Aguada Municipio,72003,-75.0,-74.0,0.0,697,327,40,59,0.0,775.674319,NaN,NaN,6.013874e+04,2020-03-16,2020-03-15 00:00:00
4,72.0,PR,Aguadilla Municipio,72005,-75.0,-74.0,0.0,1103,518,55,98,0.0,740.937443,NaN,NaN,8.579466e+03,2020-03-16,2020-03-15 00:00:00


We want to sum all the devices and completely_home_deivce of a specific county and then do the division to find out its % completely stay at home 

In [21]:
# dataframe of all counties which doesn't have shelter in place order before covid
before_no_shelter = before_covid_data[before_covid_data['days_since_shelter'].isnull()]
# dataframe of all counties which have shelter in place order before covid
before_with_shelter = before_covid_data[before_covid_data['days_since_shelter'].notnull()]

# dataframe of all counties which doesn't have shelter in place order after covid
after_no_shelter = after_covid_data[after_covid_data['days_since_shelter'].isnull()]

# dataframe of all counties which have shelter in place order after covid
after_with_shelter = after_covid_data[after_covid_data['days_since_shelter'].notnull()]

In [22]:
before_no_shelter.head()

,state,state_code,cnamelong,county_code,days_since_first_case,days_since_shelter,candidate_device_count,device_count,completely_home_device_count,part_time_work_behavior_devices,full_time_work_behavior_devices,delivery_behavior_devices,median_home_dwell_time,median_non_home_dwell_time,median_percentage_time_home,distance_traveled_from_home,Date - first case,Date - shelter in place
0,66.0,GU,Guam,66010,-75.0,NaN,0.0,310,98,41,72,0.0,395.312903,NaN,NaN,3.722158e+06,2020-03-16,NaN
1,69.0,MP,Saipan Municipality,69110,-90.0,NaN,0.0,303,121,33,35,0.0,779.036304,NaN,NaN,NaN,2020-03-31,NaN
69,78.0,VI,St. Croix Island,78010,-75.0,NaN,0.0,925,446,60,113,0.0,491.272432,NaN,NaN,2.327392e+05,2020-03-16,NaN
70,78.0,VI,St. Thomas Island,78030,-75.0,NaN,0.0,870,367,59,92,0.0,483.601149,NaN,NaN,1.976535e+05,2020-03-16,NaN
1279,5.0,AR,Arkansas County,5001,-89.0,NaN,0.0,1343,349,69,208,0.0,773.294118,NaN,NaN,7.323903e+03,2020-03-30,NaN


In [23]:
before_no_shelter = before_no_shelter.groupby(['state', 'state_code', 'cnamelong', 'county_code']).agg(
    device_count = ('device_count', 'sum'),
    completely_home_device_count = ('completely_home_device_count', 'sum')
).reset_index()

In [24]:
before_with_shelter = before_with_shelter.groupby(['state', 'state_code', 'cnamelong', 'county_code']).agg(
    device_count = ('device_count', 'sum'),
    completely_home_device_count = ('completely_home_device_count', 'sum')
).reset_index()

In [25]:
after_no_shelter = after_no_shelter.groupby(['state', 'state_code', 'cnamelong', 'county_code']).agg(
    device_count = ('device_count', 'sum'),
    completely_home_device_count = ('completely_home_device_count', 'sum')
).reset_index()

In [26]:
after_with_shelter = after_with_shelter.groupby(['state', 'state_code', 'cnamelong', 'county_code']).agg(
    device_count = ('device_count', 'sum'),
    completely_home_device_count = ('completely_home_device_count', 'sum')
).reset_index()

In [27]:
before_no_shelter.shape

(418, 6)

In [28]:
after_no_shelter.shape

(397, 6)

In [29]:
before_with_shelter.shape

(2536, 6)

In [30]:
after_with_shelter.shape

(2487, 6)

In [31]:
for county in set(before_no_shelter['cnamelong']):
  if county not in set(after_no_shelter['cnamelong']):
    print(county)

Gosper County
Chase County
Coal County
Bottineau County
Lyman County
Foster County
Ida County
Thayer County
Richardson County
Nuckolls County
Traill County
Palo Alto County
Stanley County
Harlan County
Weston County
Benson County
Tripp County
Ringgold County
Little River County
Lucas County


In [33]:
data[data['cnamelong'] == 'Palo Alto County'].head()

,date_range_start,state,state_code,cnamelong,county_code,days_since_first_case,days_since_shelter,candidate_device_count,device_count,completely_home_device_count,part_time_work_behavior_devices,full_time_work_behavior_devices,delivery_behavior_devices,median_home_dwell_time,median_non_home_dwell_time,median_percentage_time_home,distance_traveled_from_home,Date - first case,Date - shelter in place
1554,2020-01-01T00:00:00-06:00,19.0,IA,Palo Alto County,19147,-128.0,NaN,0.0,789,289,25,101,0.0,777.197719,NaN,NaN,14220.435995,2020-05-08,NaN
4567,2020-01-02T00:00:00-06:00,19.0,IA,Palo Alto County,19147,-127.0,NaN,0.0,787,214,82,152,0.0,739.599746,NaN,NaN,11351.778907,2020-05-08,NaN
7565,2020-01-03T00:00:00-06:00,19.0,IA,Palo Alto County,19147,-126.0,NaN,0.0,778,189,77,150,0.0,754.776350,NaN,NaN,9582.122108,2020-05-08,NaN
10564,2020-01-04T00:00:00-06:00,19.0,IA,Palo Alto County,19147,-125.0,NaN,0.0,775,239,71,131,0.0,806.305806,NaN,NaN,16039.590968,2020-05-08,NaN
13558,2020-01-05T00:00:00-06:00,19.0,IA,Palo Alto County,19147,-124.0,NaN,0.0,784,269,53,108,0.0,749.345663,NaN,NaN,9414.005102,2020-05-08,NaN


As we can see, some counties don't have first case until May! So after_first_case data and before_first_case data are imbalanced. So we will only use counties where both before and after data are observed.

In [34]:
before_no_shelter.head()

,state,state_code,cnamelong,county_code,device_count,completely_home_device_count
0,5.0,AR,Arkansas County,5001,121473,22952
1,5.0,AR,Ashley County,5003,135828,25603
2,5.0,AR,Baxter County,5005,201367,52327
3,5.0,AR,Benton County,5007,1673414,348702
4,5.0,AR,Boone County,5009,229563,47069


In [35]:
before_no_shelter['completely_home_device_percent'] = \
  before_no_shelter['completely_home_device_count'] / before_no_shelter['device_count']

In [36]:
before_no_shelter.head()

,state,state_code,cnamelong,county_code,device_count,completely_home_device_count,completely_home_device_percent
0,5.0,AR,Arkansas County,5001,121473,22952,0.188947
1,5.0,AR,Ashley County,5003,135828,25603,0.188496
2,5.0,AR,Baxter County,5005,201367,52327,0.259859
3,5.0,AR,Benton County,5007,1673414,348702,0.208378
4,5.0,AR,Boone County,5009,229563,47069,0.205037


In [37]:
before_no_shelter.drop(['device_count', 'completely_home_device_count'], axis = 1,
                       inplace = True)

In [38]:
before_no_shelter.head()

,state,state_code,cnamelong,county_code,completely_home_device_percent
0,5.0,AR,Arkansas County,5001,0.188947
1,5.0,AR,Ashley County,5003,0.188496
2,5.0,AR,Baxter County,5005,0.259859
3,5.0,AR,Benton County,5007,0.208378
4,5.0,AR,Boone County,5009,0.205037


In [39]:
after_no_shelter.head()

,state,state_code,cnamelong,county_code,device_count,completely_home_device_count
0,5.0,AR,Arkansas County,5001,52657,12686
1,5.0,AR,Ashley County,5003,61736,16349
2,5.0,AR,Baxter County,5005,94854,32820
3,5.0,AR,Benton County,5007,936427,341644
4,5.0,AR,Boone County,5009,126588,38616


In [40]:
after_no_shelter['completely_home_device_percent'] = \
  after_no_shelter['completely_home_device_count'] / after_no_shelter['device_count']
after_no_shelter.drop(['device_count', 'completely_home_device_count'], axis = 1,
                       inplace = True)

In [41]:
after_no_shelter.head()

,state,state_code,cnamelong,county_code,completely_home_device_percent
0,5.0,AR,Arkansas County,5001,0.240918
1,5.0,AR,Ashley County,5003,0.264821
2,5.0,AR,Baxter County,5005,0.346005
3,5.0,AR,Benton County,5007,0.364838
4,5.0,AR,Boone County,5009,0.305053


In [42]:
merged_no_shelter = before_no_shelter.merge(after_no_shelter, on=['state', 'state_code',\
                                                                  'cnamelong', 'county_code'],\
                                            suffixes = ('_before', '_after'))

In [43]:
merged_no_shelter.head()

,state,state_code,cnamelong,county_code,completely_home_device_percent_before,completely_home_device_percent_after
0,5.0,AR,Arkansas County,5001,0.188947,0.240918
1,5.0,AR,Ashley County,5003,0.188496,0.264821
2,5.0,AR,Baxter County,5005,0.259859,0.346005
3,5.0,AR,Benton County,5007,0.208378,0.364838
4,5.0,AR,Boone County,5009,0.205037,0.305053


In [44]:
merged_no_shelter['%diff_stay_at_home'] = merged_no_shelter['completely_home_device_percent_after'] -  merged_no_shelter['completely_home_device_percent_before']

In [45]:
merged_no_shelter.head()

,state,state_code,cnamelong,county_code,completely_home_device_percent_before,completely_home_device_percent_after,%diff_stay_at_home
0,5.0,AR,Arkansas County,5001,0.188947,0.240918,0.051970
1,5.0,AR,Ashley County,5003,0.188496,0.264821,0.076325
2,5.0,AR,Baxter County,5005,0.259859,0.346005,0.086147
3,5.0,AR,Benton County,5007,0.208378,0.364838,0.156460
4,5.0,AR,Boone County,5009,0.205037,0.305053,0.100015


In [46]:
merged_no_shelter.drop(['completely_home_device_percent_before', 'completely_home_device_percent_after'], axis = 1,
                       inplace = True)

In [47]:
merged_no_shelter.head()

,state,state_code,cnamelong,county_code,%diff_stay_at_home
0,5.0,AR,Arkansas County,5001,0.051970
1,5.0,AR,Ashley County,5003,0.076325
2,5.0,AR,Baxter County,5005,0.086147
3,5.0,AR,Benton County,5007,0.156460
4,5.0,AR,Boone County,5009,0.100015


In [48]:
before_with_shelter['completely_home_device_percent'] = \
  before_with_shelter['completely_home_device_count'] / before_with_shelter['device_count']
before_with_shelter.drop(['device_count', 'completely_home_device_count'], axis = 1,
                       inplace = True)

after_with_shelter['completely_home_device_percent'] = \
  after_with_shelter['completely_home_device_count'] / after_with_shelter['device_count']
after_with_shelter.drop(['device_count', 'completely_home_device_count'], axis = 1,
                       inplace = True)
merged_with_shelter = before_with_shelter.merge(after_with_shelter, on=['state', 'state_code',\
                                                                  'cnamelong', 'county_code'],\
                                            suffixes = ('_before', '_after'))

merged_with_shelter['%diff_stay_at_home'] = merged_with_shelter['completely_home_device_percent_after'] - merged_with_shelter['completely_home_device_percent_before']

merged_with_shelter.drop(['completely_home_device_percent_before', 'completely_home_device_percent_after'], axis = 1,
                       inplace = True)

In [49]:
merged_with_shelter.head()

,state,state_code,cnamelong,county_code,%diff_stay_at_home
0,1.0,AL,Autauga County,1001,0.119839
1,1.0,AL,Baldwin County,1003,0.112319
2,1.0,AL,Barbour County,1005,0.071000
3,1.0,AL,Bibb County,1007,0.089074
4,1.0,AL,Blount County,1009,0.098240


In [50]:
merged_no_shelter.head()

,state,state_code,cnamelong,county_code,%diff_stay_at_home
0,5.0,AR,Arkansas County,5001,0.051970
1,5.0,AR,Ashley County,5003,0.076325
2,5.0,AR,Baxter County,5005,0.086147
3,5.0,AR,Benton County,5007,0.156460
4,5.0,AR,Boone County,5009,0.100015


In [51]:
merged_no_shelter.shape

(395, 5)

In [52]:
merged_with_shelter.shape

(2484, 5)

In [53]:
merged_with_shelter['%diff_stay_at_home'].mean() - merged_no_shelter['%diff_stay_at_home'].mean()

0.028664851038466707

In [54]:
# SIP means does this county has issued shelter in place?
# 1 means yes, 0 means no
merged_with_shelter['SIP?'] = 1 

In [55]:
merged_with_shelter.head()

,state,state_code,cnamelong,county_code,%diff_stay_at_home,SIP?
0,1.0,AL,Autauga County,1001,0.119839,1
1,1.0,AL,Baldwin County,1003,0.112319,1
2,1.0,AL,Barbour County,1005,0.071000,1
3,1.0,AL,Bibb County,1007,0.089074,1
4,1.0,AL,Blount County,1009,0.098240,1


In [56]:
merged_no_shelter['SIP?'] = 0

In [57]:
merged_no_shelter.head()

,state,state_code,cnamelong,county_code,%diff_stay_at_home,SIP?
0,5.0,AR,Arkansas County,5001,0.051970,0
1,5.0,AR,Ashley County,5003,0.076325,0
2,5.0,AR,Baxter County,5005,0.086147,0
3,5.0,AR,Benton County,5007,0.156460,0
4,5.0,AR,Boone County,5009,0.100015,0


In [58]:
merged_data = merged_with_shelter.append(merged_no_shelter)

In [59]:
merged_data.shape

(2879, 6)

In [60]:
merged_data.head()

,state,state_code,cnamelong,county_code,%diff_stay_at_home,SIP?
0,1.0,AL,Autauga County,1001,0.119839,1
1,1.0,AL,Baldwin County,1003,0.112319,1
2,1.0,AL,Barbour County,1005,0.071000,1
3,1.0,AL,Bibb County,1007,0.089074,1
4,1.0,AL,Blount County,1009,0.098240,1


In [61]:
merged_data[merged_data['SIP?'] == 0]

,state,state_code,cnamelong,county_code,%diff_stay_at_home,SIP?
0,5.0,AR,Arkansas County,5001,0.051970,0
1,5.0,AR,Ashley County,5003,0.076325,0
2,5.0,AR,Baxter County,5005,0.086147,0
3,5.0,AR,Benton County,5007,0.156460,0
4,5.0,AR,Boone County,5009,0.100015,0
...,...,...,...,...,...,...
390,56.0,WY,Washakie County,56043,0.083395,0
391,66.0,GU,Guam,66010,0.208088,0
392,69.0,MP,Saipan Municipality,69110,0.176838,0
393,78.0,VI,St. Croix Island,78010,0.079552,0


In [62]:
merged_data[merged_data['SIP?'] == 1]

,state,state_code,cnamelong,county_code,%diff_stay_at_home,SIP?
0,1.0,AL,Autauga County,1001,0.119839,1
1,1.0,AL,Baldwin County,1003,0.112319,1
2,1.0,AL,Barbour County,1005,0.071000,1
3,1.0,AL,Bibb County,1007,0.089074,1
4,1.0,AL,Blount County,1009,0.098240,1
...,...,...,...,...,...,...
2479,72.0,PR,Vega Baja Municipio,72145,0.284883,1
2480,72.0,PR,Vieques Municipio,72147,0.195316,1
2481,72.0,PR,Villalba Municipio,72149,0.251756,1
2482,72.0,PR,Yabucoa Municipio,72151,0.230393,1


In [63]:
merged_data.to_csv("county_data_with_and_without_SIP.csv", index = False)

# Add covariates like median income and political difference of a county to existing data.

In [ ]:
cd drive/My\ Drive/week5

/content/drive/My Drive/week5


In [64]:
import pandas as pd
import numpy as np

In [65]:
data = pd.read_csv("county_data_with_and_without_SIP.csv")

In [67]:
# covariates data
df_chr_1 = pd.read_csv('2020 County Health Rankings Data - Ranked Measure Data.csv')
df_chr_2 = pd.read_csv('2020 County Health Rankings Data - Additional Measure Data.csv')
df_governors = pd.read_csv('us_states_governors.csv', encoding='latin-1')
df_election = pd.read_csv('2016_US_County_Level_Presidential_Results.csv')

In [68]:
df_election['political_party'] = df_election.apply(lambda x: 'Republican' if x['per_gop'] > x['per_dem'] else 'Democratic', axis=1)
df_election['political_diff'] = df_election.apply(lambda x: x['per_dem'] - x['per_gop'], axis=1)

In [69]:
df_chr = df_chr_1.merge(df_chr_2, on=['FIPS', 'State', 'County'])
columns = ['FIPS', 'State', 'County', 'Population_y', 'Years of Potential Life Lost Rate', '% Fair or Poor Health', 
           'Average Number of Physically Unhealthy Days', 'Average Number of Mentally Unhealthy Days',
           '% Low Birthweight', '% Smokers', '% Adults with Obesity', 'Food Environment Index', 
           '% Physically Inactive', '% With Access to Exercise Opportunities', '% Excessive Drinking',
           '% Driving Deaths with Alcohol Involvement', 'Chlamydia Rate', 'Teen Birth Rate', '% Uninsured_x',
           'Primary Care Physicians Rate', 'Primary Care Physicians Ratio', 
           'Dentist Rate', 'Dentist Ratio', 'Mental Health Provider Rate', 'Mental Health Provider Ratio',
           'Preventable Hospitalization Rate', '% With Annual Mammogram', '% Vaccinated',
           'High School Graduation Rate', '% Some College', '% Unemployed', '% Children in Poverty',
           'Income Ratio', '% Single-Parent Households', 'Social Association Rate', 'Violent Crime Rate',
           'Injury Death Rate', 'Average Daily PM2.5', 'Presence of Water Violation', '% Severe Housing Problems',
           '% Drive Alone to Work', '% Long Commute - Drives Alone',
           'Life Expectancy', 'Age-Adjusted Death Rate', 'Child Mortality Rate',
           'Infant Mortality Rate', '% Frequent Physical Distress', '% Frequent Mental Distress',
           '% Adults with Diabetes', 'HIV Prevalence Rate', 
           '% Food Insecure', '% Limited Access to Healthy Foods',
           'Drug Overdose Mortality Rate', 'Motor Vehicle Mortality Rate',
           '% Insufficient Sleep', '% Uninsured_y', '% Uninsured.1',
           'Other Primary Care Provider Rate', 'Other Primary Care Provider Ratio','% Disconnected Youth',
           'Average Grade Performance', 'Average Grade Performance.1', 'Median Household Income', 
           '% Enrolled in Free or Reduced Lunch', 'Segregation index', 'Segregation Index', 'Homicide Rate',
           'Suicide Rate (Age-Adjusted)', 'Firearm Fatalities Rate',
           'Juvenile Arrest Rate', 'Average Traffic Volume per Meter of Major Roadways',
           '% Homeowners', '% Severe Housing Cost Burden', '% less than 18 years of age', '% 65 and over',
           '% Black', '% American Indian & Alaska Native', '% Asian', '% Native Hawaiian/Other Pacific Islander',
           '% Hispanic', '% Non-Hispanic White', '% Not Proficient in English', '% Female', '% Rural'
          ]
df_chr = df_chr[columns]

In [70]:
df_governors.head()

,State Code,State Name,Governor,Party,Date,Shelter at home begins,Shelter in place ends,Reopen
0,AL,Alabama,Kay Ivey,Republican,10-Apr-17,4/4/2020,4/30/2020,4/30/2020
1,AK,Alaska,Mike Dunleavy,Republican,3-Dec-18,3/28/2020,4/24/2020,4/24/2020
2,AS,American Samoa,Lolo Matalasi Moliga,Independent,3-Jan-13,NaN,NaN,NaN
3,AZ,Arizona,Doug Ducey,Republican,5-Jan-15,3/31/2020,5/16/2020,5/8/2020
4,AR,Arkansas,Asa Hutchinson,Republican,13-Jan-15,NaN,NaN,NaN


In [71]:
df_election.head()

,Unnamed: 0,votes_dem,votes_gop,total_votes,per_dem,per_gop,diff,per_point_diff,state_abbr,county_name,combined_fips,political_party,political_diff
0,0,93003.0,130413.0,246588.0,0.377159,0.52887,"37,410",15.17%,AK,Alaska,2013,Republican,-0.151711
1,1,93003.0,130413.0,246588.0,0.377159,0.52887,"37,410",15.17%,AK,Alaska,2016,Republican,-0.151711
2,2,93003.0,130413.0,246588.0,0.377159,0.52887,"37,410",15.17%,AK,Alaska,2020,Republican,-0.151711
3,3,93003.0,130413.0,246588.0,0.377159,0.52887,"37,410",15.17%,AK,Alaska,2050,Republican,-0.151711
4,4,93003.0,130413.0,246588.0,0.377159,0.52887,"37,410",15.17%,AK,Alaska,2060,Republican,-0.151711


In [72]:
df_chr.head()

,FIPS,State,County,Population_y,Years of Potential Life Lost Rate,% Fair or Poor Health,Average Number of Physically Unhealthy Days,Average Number of Mentally Unhealthy Days,% Low Birthweight,% Smokers,% Adults with Obesity,Food Environment Index,% Physically Inactive,% With Access to Exercise Opportunities,% Excessive Drinking,% Driving Deaths with Alcohol Involvement,Chlamydia Rate,Teen Birth Rate,% Uninsured_x,Primary Care Physicians Rate,Primary Care Physicians Ratio,Dentist Rate,Dentist Ratio,Mental Health Provider Rate,Mental Health Provider Ratio,Preventable Hospitalization Rate,% With Annual Mammogram,% Vaccinated,High School Graduation Rate,% Some College,% Unemployed,% Children in Poverty,Income Ratio,% Single-Parent Households,Social Association Rate,Violent Crime Rate,Injury Death Rate,Average Daily PM2.5,Presence of Water Violation,% Severe Housing Problems,...,Child Mortality Rate,Infant Mortality Rate,% Frequent Physical Distress,% Frequent Mental Distress,% Adults with Diabetes,HIV Prevalence Rate,% Food Insecure,% Limited Access to Healthy Foods,Drug Overdose Mortality Rate,Motor Vehicle Mortality Rate,% Insufficient Sleep,% Uninsured_y,% Uninsured.1,Other Primary Care Provider Rate,Other Primary Care Provider Ratio,% Disconnected Youth,Average Grade Performance,Average Grade Performance.1,Median Household Income,% Enrolled in Free or Reduced Lunch,Segregation index,Segregation Index,Homicide Rate,Suicide Rate (Age-Adjusted),Firearm Fatalities Rate,Juvenile Arrest Rate,Average Traffic Volume per Meter of Major Roadways,% Homeowners,% Severe Housing Cost Burden,% less than 18 years of age,% 65 and over,% Black,% American Indian & Alaska Native,% Asian,% Native Hawaiian/Other Pacific Islander,% Hispanic,% Non-Hispanic White,% Not Proficient in English,% Female,% Rural
0,1000,Alabama,NaN,4887871,9943.0,22,4.9,4.9,10.0,21,36,5.8,30,61.0,14,28.0,614.1,31.0,11.0,65.0,1543:1,48.0,2076:1,101.0,988:1,5805.0,40.0,41.0,89.0,60,3.9,24.0,5.3,37.0,12.4,480.0,82.0,11.0,NaN,14,...,69.0,8.0,16,16,14,310.0,16,8.0,16.0,20.0,38,14.0,3.0,86.0,1162:1,8.0,3.0,2.8,49881.0,56.0,57.0,51.0,10.0,16.0,21.0,NaN,166,69,13.0,22.3,16.9,26.5,0.7,1.5,0.1,4.4,65.4,1,51.6,41.0
1,1001,Alabama,Autauga,55601,8129.0,21,4.7,4.7,9.0,18,33,7.2,35,69.0,15,27.0,407.2,25.0,9.0,45.0,2220:1,32.0,3089:1,23.0,4277:1,7108.0,41.0,41.0,90.0,62,3.6,19.0,5.2,26.0,12.1,272.0,69.0,11.7,No,15,...,56.0,8.0,14,14,11,219.0,13,12.0,11.0,19.0,36,11.0,3.0,40.0,2527:1,NaN,3.0,2.8,59338.0,43.0,25.0,24.0,5.0,18.0,16.0,11.0,88,75,13.0,23.7,15.6,19.3,0.5,1.2,0.1,3.0,74.3,1,51.4,42.0
2,1003,Alabama,Baldwin,218022,7354.0,18,4.2,4.3,8.0,17,31,8.0,27,74.0,18,31.0,325.0,28.0,11.0,73.0,1372:1,50.0,2019:1,96.0,1038:1,4041.0,43.0,44.0,86.0,67,3.6,14.0,4.4,24.0,10.2,204.0,72.0,10.3,No,14,...,47.0,6.0,12,13,11,167.0,12,5.0,14.0,16.0,33,14.0,4.0,56.0,1787:1,8.0,3.0,2.9,57588.0,48.0,41.0,32.0,3.0,19.0,14.0,26.0,87,74,12.0,21.6,20.4,8.8,0.8,1.2,0.1,4.6,83.1,1,51.5,42.3
3,1005,Alabama,Barbour,24881,10254.0,30,5.4,5.2,11.0,22,42,5.6,24,53.0,13,40.0,716.3,41.0,12.0,32.0,3159:1,36.0,2765:1,8.0,12441:1,6209.0,45.0,37.0,81.0,35,5.2,44.0,5.7,57.0,7.5,414.0,70.0,11.5,No,15,...,74.0,NaN,17,17,18,421.0,22,11.0,NaN,23.0,39,16.0,3.0,52.0,1914:1,13.0,2.7,2.4,34382.0,63.0,25.0,23.0,8.0,13.0,18.0,15.0,102,61,14.0,20.9,19.4,48.0,0.7,0.5,0.2,4.3,45.6,2,47.2,67.8
4,1007,Alabama,Bibb,22400,11978.0,19,4.6,4.6,10.0,19,38,7.8,34,16.0,16,28.0,339.7,42.0,10.0,49.0,2061:1,22.0,4480:1,22.0,4480:1,5961.0,40.0,38.0,84.0,44,4.0,28.0,4.4,29.0,8.4,89.0,108.0,11.2,No,10,...,86.0,12.0,14,14,15,197.0,14,3.0,28.0,26.0,38,13.0,3.0,112.0,896:1,NaN,2.6,2.4,46064.0,62.0,53.0,53.0,8.0,21.0,24.0,NaN,29,75,10.0,20.5,16.5,21.1,0.4,0.2,0.1,2.6,74.6,0,46.8,68.4


In [73]:
df_merged = data.merge(df_governors, how='left', left_on='state_code', right_on='State Code')
df_merged = df_merged.merge(df_chr, left_on='county_code', right_on='FIPS')
df_merged = df_merged.merge(df_election, left_on='county_code', right_on='combined_fips')
df_merged.head()

,state,state_code,cnamelong,county_code,%diff_stay_at_home,SIP?,State Code,State Name,Governor,Party,Date,Shelter at home begins,Shelter in place ends,Reopen,FIPS,State,County,Population_y,Years of Potential Life Lost Rate,% Fair or Poor Health,Average Number of Physically Unhealthy Days,Average Number of Mentally Unhealthy Days,% Low Birthweight,% Smokers,% Adults with Obesity,Food Environment Index,% Physically Inactive,% With Access to Exercise Opportunities,% Excessive Drinking,% Driving Deaths with Alcohol Involvement,Chlamydia Rate,Teen Birth Rate,% Uninsured_x,Primary Care Physicians Rate,Primary Care Physicians Ratio,Dentist Rate,Dentist Ratio,Mental Health Provider Rate,Mental Health Provider Ratio,Preventable Hospitalization Rate,...,Other Primary Care Provider Rate,Other Primary Care Provider Ratio,% Disconnected Youth,Average Grade Performance,Average Grade Performance.1,Median Household Income,% Enrolled in Free or Reduced Lunch,Segregation index,Segregation Index,Homicide Rate,Suicide Rate (Age-Adjusted),Firearm Fatalities Rate,Juvenile Arrest Rate,Average Traffic Volume per Meter of Major Roadways,% Homeowners,% Severe Housing Cost Burden,% less than 18 years of age,% 65 and over,% Black,% American Indian & Alaska Native,% Asian,% Native Hawaiian/Other Pacific Islander,% Hispanic,% Non-Hispanic White,% Not Proficient in English,% Female,% Rural,Unnamed: 0,votes_dem,votes_gop,total_votes,per_dem,per_gop,diff,per_point_diff,state_abbr,county_name,combined_fips,political_party,political_diff
0,1.0,AL,Autauga County,1001,0.119839,1,AL,Alabama,Kay Ivey,Republican,10-Apr-17,4/4/2020,4/30/2020,4/30/2020,1001,Alabama,Autauga,55601,8129.0,21,4.7,4.7,9.0,18,33,7.2,35,69.0,15,27.0,407.2,25.0,9.0,45.0,2220:1,32.0,3089:1,23.0,4277:1,7108.0,...,40.0,2527:1,NaN,3.0,2.8,59338.0,43.0,25.0,24.0,5.0,18.0,16.0,11.0,88,75,13.0,23.7,15.6,19.3,0.5,1.2,0.1,3.0,74.3,1,51.4,42.0,29,5908.0,18110.0,24661.0,0.239569,0.734358,"12,202",49.48%,AL,Autauga County,1001,Republican,-0.494789
1,1.0,AL,Baldwin County,1003,0.112319,1,AL,Alabama,Kay Ivey,Republican,10-Apr-17,4/4/2020,4/30/2020,4/30/2020,1003,Alabama,Baldwin,218022,7354.0,18,4.2,4.3,8.0,17,31,8.0,27,74.0,18,31.0,325.0,28.0,11.0,73.0,1372:1,50.0,2019:1,96.0,1038:1,4041.0,...,56.0,1787:1,8.0,3.0,2.9,57588.0,48.0,41.0,32.0,3.0,19.0,14.0,26.0,87,74,12.0,21.6,20.4,8.8,0.8,1.2,0.1,4.6,83.1,1,51.5,42.3,30,18409.0,72780.0,94090.0,0.195653,0.773515,"54,371",57.79%,AL,Baldwin County,1003,Republican,-0.577862
2,1.0,AL,Barbour County,1005,0.071000,1,AL,Alabama,Kay Ivey,Republican,10-Apr-17,4/4/2020,4/30/2020,4/30/2020,1005,Alabama,Barbour,24881,10254.0,30,5.4,5.2,11.0,22,42,5.6,24,53.0,13,40.0,716.3,41.0,12.0,32.0,3159:1,36.0,2765:1,8.0,12441:1,6209.0,...,52.0,1914:1,13.0,2.7,2.4,34382.0,63.0,25.0,23.0,8.0,13.0,18.0,15.0,102,61,14.0,20.9,19.4,48.0,0.7,0.5,0.2,4.3,45.6,2,47.2,67.8,31,4848.0,5431.0,10390.0,0.466603,0.522714,583,5.61%,AL,Barbour County,1005,Republican,-0.056112
3,1.0,AL,Bibb County,1007,0.089074,1,AL,Alabama,Kay Ivey,Republican,10-Apr-17,4/4/2020,4/30/2020,4/30/2020,1007,Alabama,Bibb,22400,11978.0,19,4.6,4.6,10.0,19,38,7.8,34,16.0,16,28.0,339.7,42.0,10.0,49.0,2061:1,22.0,4480:1,22.0,4480:1,5961.0,...,112.0,896:1,NaN,2.6,2.4,46064.0,62.0,53.0,53.0,8.0,21.0,24.0,NaN,29,75,10.0,20.5,16.5,21.1,0.4,0.2,0.1,2.6,74.6,0,46.8,68.4,32,1874.0,6733.0,8748.0,0.214220,0.769662,"4,859",55.54%,AL,Bibb County,1007,Republican,-0.555441
4,1.0,AL,Blount County,1009,0.098240,1,AL,Alabama,Kay Ivey,Republican,10-Apr-17,4/4/2020,4/30/2020,4/30/2020,1009,Alabama,Blount,57840,11335.0,22,4.9,4.9,8.0,19,34,8.4,30,16.0,14,19.0,234.4,34.0,13.0,22.0,4463:1,19.0,5258:1,16.0,6427:1,4060.0,...,22.0,4449:1,19.0,3.0,2.8,50412.0,53.0,48.0,18.0,6.0,17.0,20.0,7.0,33,79,8.0,23.2,18.2,1.5,0.7,0.3,0.1,9.6,86.9,2,50.7,90.0,33,2150.0,22808.0,25384.0,0.084699,0.898519,"20,658",81.38%,AL,Blount County,1009,Republican,-0.813820


In [74]:
df_merged.shape

(2799, 111)

In [75]:
data.shape

(2879, 6)

we are losing 80 rows after merging with covariates, let's see why

In [76]:
for county_code in set(data['county_code']):
  if not county_code in set(df_chr['FIPS']):
    print(county_code)

66010
69110
78010
78030
72001
72003
72005
72007
72009
72011
72013
72015
72017
72019
72021
72023
72025
72027
72029
72031
72033
72035
72037
72039
72041
72043
72045
72047
72051
72053
72054
72055
72057
72059
72061
72063
72065
72067
72069
72071
72073
72075
72077
72079
72081
72083
72085
72087
72089
72091
72093
72095
72097
72099
72101
72103
72105
72107
72109
72111
72113
72115
72117
72119
72121
72123
72125
72127
72129
72131
72133
72135
72137
72139
72141
72143
72145
72147
72149
72151
72153


In [77]:
df_chr[df_chr['FIPS'] == 66010]

,FIPS,State,County,Population_y,Years of Potential Life Lost Rate,% Fair or Poor Health,Average Number of Physically Unhealthy Days,Average Number of Mentally Unhealthy Days,% Low Birthweight,% Smokers,% Adults with Obesity,Food Environment Index,% Physically Inactive,% With Access to Exercise Opportunities,% Excessive Drinking,% Driving Deaths with Alcohol Involvement,Chlamydia Rate,Teen Birth Rate,% Uninsured_x,Primary Care Physicians Rate,Primary Care Physicians Ratio,Dentist Rate,Dentist Ratio,Mental Health Provider Rate,Mental Health Provider Ratio,Preventable Hospitalization Rate,% With Annual Mammogram,% Vaccinated,High School Graduation Rate,% Some College,% Unemployed,% Children in Poverty,Income Ratio,% Single-Parent Households,Social Association Rate,Violent Crime Rate,Injury Death Rate,Average Daily PM2.5,Presence of Water Violation,% Severe Housing Problems,...,Child Mortality Rate,Infant Mortality Rate,% Frequent Physical Distress,% Frequent Mental Distress,% Adults with Diabetes,HIV Prevalence Rate,% Food Insecure,% Limited Access to Healthy Foods,Drug Overdose Mortality Rate,Motor Vehicle Mortality Rate,% Insufficient Sleep,% Uninsured_y,% Uninsured.1,Other Primary Care Provider Rate,Other Primary Care Provider Ratio,% Disconnected Youth,Average Grade Performance,Average Grade Performance.1,Median Household Income,% Enrolled in Free or Reduced Lunch,Segregation index,Segregation Index,Homicide Rate,Suicide Rate (Age-Adjusted),Firearm Fatalities Rate,Juvenile Arrest Rate,Average Traffic Volume per Meter of Major Roadways,% Homeowners,% Severe Housing Cost Burden,% less than 18 years of age,% 65 and over,% Black,% American Indian & Alaska Native,% Asian,% Native Hawaiian/Other Pacific Islander,% Hispanic,% Non-Hispanic White,% Not Proficient in English,% Female,% Rural


It turns out we don't have covariate data in our df_chr on some of the counties. (around 84 of them). For now we will just ignore these data. Cause without covariates we can't do the propensity matching business

In [78]:
df_merged.to_csv("county_data_with_covariates.csv", index = False)

Now let's pick only the covariates that we are using for propensity score matching:


1.   Median Income, % Rural, population density, political difference
2.   % of 18 or less, % 65 and older
3.   % Asian, % Black
4.   first covid case date



In [79]:
df_merged.head()

,state,state_code,cnamelong,county_code,%diff_stay_at_home,SIP?,State Code,State Name,Governor,Party,Date,Shelter at home begins,Shelter in place ends,Reopen,FIPS,State,County,Population_y,Years of Potential Life Lost Rate,% Fair or Poor Health,Average Number of Physically Unhealthy Days,Average Number of Mentally Unhealthy Days,% Low Birthweight,% Smokers,% Adults with Obesity,Food Environment Index,% Physically Inactive,% With Access to Exercise Opportunities,% Excessive Drinking,% Driving Deaths with Alcohol Involvement,Chlamydia Rate,Teen Birth Rate,% Uninsured_x,Primary Care Physicians Rate,Primary Care Physicians Ratio,Dentist Rate,Dentist Ratio,Mental Health Provider Rate,Mental Health Provider Ratio,Preventable Hospitalization Rate,...,Other Primary Care Provider Rate,Other Primary Care Provider Ratio,% Disconnected Youth,Average Grade Performance,Average Grade Performance.1,Median Household Income,% Enrolled in Free or Reduced Lunch,Segregation index,Segregation Index,Homicide Rate,Suicide Rate (Age-Adjusted),Firearm Fatalities Rate,Juvenile Arrest Rate,Average Traffic Volume per Meter of Major Roadways,% Homeowners,% Severe Housing Cost Burden,% less than 18 years of age,% 65 and over,% Black,% American Indian & Alaska Native,% Asian,% Native Hawaiian/Other Pacific Islander,% Hispanic,% Non-Hispanic White,% Not Proficient in English,% Female,% Rural,Unnamed: 0,votes_dem,votes_gop,total_votes,per_dem,per_gop,diff,per_point_diff,state_abbr,county_name,combined_fips,political_party,political_diff
0,1.0,AL,Autauga County,1001,0.119839,1,AL,Alabama,Kay Ivey,Republican,10-Apr-17,4/4/2020,4/30/2020,4/30/2020,1001,Alabama,Autauga,55601,8129.0,21,4.7,4.7,9.0,18,33,7.2,35,69.0,15,27.0,407.2,25.0,9.0,45.0,2220:1,32.0,3089:1,23.0,4277:1,7108.0,...,40.0,2527:1,NaN,3.0,2.8,59338.0,43.0,25.0,24.0,5.0,18.0,16.0,11.0,88,75,13.0,23.7,15.6,19.3,0.5,1.2,0.1,3.0,74.3,1,51.4,42.0,29,5908.0,18110.0,24661.0,0.239569,0.734358,"12,202",49.48%,AL,Autauga County,1001,Republican,-0.494789
1,1.0,AL,Baldwin County,1003,0.112319,1,AL,Alabama,Kay Ivey,Republican,10-Apr-17,4/4/2020,4/30/2020,4/30/2020,1003,Alabama,Baldwin,218022,7354.0,18,4.2,4.3,8.0,17,31,8.0,27,74.0,18,31.0,325.0,28.0,11.0,73.0,1372:1,50.0,2019:1,96.0,1038:1,4041.0,...,56.0,1787:1,8.0,3.0,2.9,57588.0,48.0,41.0,32.0,3.0,19.0,14.0,26.0,87,74,12.0,21.6,20.4,8.8,0.8,1.2,0.1,4.6,83.1,1,51.5,42.3,30,18409.0,72780.0,94090.0,0.195653,0.773515,"54,371",57.79%,AL,Baldwin County,1003,Republican,-0.577862
2,1.0,AL,Barbour County,1005,0.071000,1,AL,Alabama,Kay Ivey,Republican,10-Apr-17,4/4/2020,4/30/2020,4/30/2020,1005,Alabama,Barbour,24881,10254.0,30,5.4,5.2,11.0,22,42,5.6,24,53.0,13,40.0,716.3,41.0,12.0,32.0,3159:1,36.0,2765:1,8.0,12441:1,6209.0,...,52.0,1914:1,13.0,2.7,2.4,34382.0,63.0,25.0,23.0,8.0,13.0,18.0,15.0,102,61,14.0,20.9,19.4,48.0,0.7,0.5,0.2,4.3,45.6,2,47.2,67.8,31,4848.0,5431.0,10390.0,0.466603,0.522714,583,5.61%,AL,Barbour County,1005,Republican,-0.056112
3,1.0,AL,Bibb County,1007,0.089074,1,AL,Alabama,Kay Ivey,Republican,10-Apr-17,4/4/2020,4/30/2020,4/30/2020,1007,Alabama,Bibb,22400,11978.0,19,4.6,4.6,10.0,19,38,7.8,34,16.0,16,28.0,339.7,42.0,10.0,49.0,2061:1,22.0,4480:1,22.0,4480:1,5961.0,...,112.0,896:1,NaN,2.6,2.4,46064.0,62.0,53.0,53.0,8.0,21.0,24.0,NaN,29,75,10.0,20.5,16.5,21.1,0.4,0.2,0.1,2.6,74.6,0,46.8,68.4,32,1874.0,6733.0,8748.0,0.214220,0.769662,"4,859",55.54%,AL,Bibb County,1007,Republican,-0.555441
4,1.0,AL,Blount County,1009,0.098240,1,AL,Alabama,Kay Ivey,Republican,10-Apr-17,4/4/2020,4/30/2020,4/30/2020,1009,Alabama,Blount,57840,11335.0,22,4.9,4.9,8.0,19,34,8.4,30,16.0,14,19.0,234.4,34.0,13.0,22.0,4463:1,19.0,5258:1,16.0,6427:1,4060.0,...,22.0,4449:1,19.0,3.0,2.8,50412.0,53.0,48.0,18.0,6.0,17.0,20.0,7.0,33,79,8.0,23.2,18.2,1.5,0.7,0.3,0.1,9.6,86.9,2,50.7,90.0,33,2150.0,22808.0,25384.0,0.084699,0.898519,"20,658",81.38%,AL,Blount County,1009,Republican,-0.813820


In [80]:
# all the data we have about a county
for col in df_merged.columns:
  print(col)

state
state_code
cnamelong
county_code
%diff_stay_at_home
SIP?
State Code
State Name
Governor
Party
Date
Shelter at home begins
Shelter in place ends
Reopen
FIPS
State
County
Population_y
Years of Potential Life Lost Rate
% Fair or Poor Health
Average Number of Physically Unhealthy Days
Average Number of Mentally Unhealthy Days
% Low Birthweight
% Smokers
% Adults with Obesity
Food Environment Index
% Physically Inactive
% With Access to Exercise Opportunities
% Excessive Drinking
% Driving Deaths with Alcohol Involvement
Chlamydia Rate
Teen Birth Rate
% Uninsured_x
Primary Care Physicians Rate
Primary Care Physicians Ratio
Dentist Rate
Dentist Ratio
Mental Health Provider Rate
Mental Health Provider Ratio
Preventable Hospitalization Rate
% With Annual Mammogram
% Vaccinated
High School Graduation Rate
% Some College
% Unemployed
% Children in Poverty
Income Ratio
% Single-Parent Households
Social Association Rate
Violent Crime Rate
Injury Death Rate
Average Daily PM2.5
Presence of Wat

In [83]:
df_reduced_covariate = \
df_merged[['state', 'state_code', 'State Name','cnamelong', 'county_code', '%diff_stay_at_home',\
           'SIP?', 'Median Household Income', '% Rural', 'Population_y', 'political_diff',\
           '% less than 18 years of age', '% 65 and over', '% Asian', '% Black', '% Hispanic',\
           '% Non-Hispanic White']]

In [84]:
df_reduced_covariate.head()

,state,state_code,State Name,cnamelong,county_code,%diff_stay_at_home,SIP?,Median Household Income,% Rural,Population_y,political_diff,% less than 18 years of age,% 65 and over,% Asian,% Black,% Hispanic,% Non-Hispanic White
0,1.0,AL,Alabama,Autauga County,1001,0.119839,1,59338.0,42.0,55601,-0.494789,23.7,15.6,1.2,19.3,3.0,74.3
1,1.0,AL,Alabama,Baldwin County,1003,0.112319,1,57588.0,42.3,218022,-0.577862,21.6,20.4,1.2,8.8,4.6,83.1
2,1.0,AL,Alabama,Barbour County,1005,0.071000,1,34382.0,67.8,24881,-0.056112,20.9,19.4,0.5,48.0,4.3,45.6
3,1.0,AL,Alabama,Bibb County,1007,0.089074,1,46064.0,68.4,22400,-0.555441,20.5,16.5,0.2,21.1,2.6,74.6
4,1.0,AL,Alabama,Blount County,1009,0.098240,1,50412.0,90.0,57840,-0.813820,23.2,18.2,0.3,1.5,9.6,86.9


Let's exclude the first covid case date for now, if we want, can add it later. (don't know how logisitc regresion cope with date object...)

In [85]:
df_reduced_covariate.to_csv("county_data_with_reduced_covariates.csv", index = False)